In [1]:
import torch
import numpy as np
import pickle
from yolonet import Backbone

In [2]:
weight_path = '/home/dk/jyl/Yolo/V2/model/ckpt/only_params_trained_yolo_voc'
trained_yolo = torch.load(weight_path)

In [3]:
trained_yolo

OrderedDict([('stage1_conv1.0.weight',
              tensor([[[[-5.6318e-02,  1.5582e-02,  3.8185e-02],
                        [-1.3139e-01,  2.0700e-02,  9.7388e-02],
                        [-2.2657e-02,  3.1506e-02,  2.5467e-02]],
              
                       [[-6.5613e-02,  3.0166e-02,  7.1327e-02],
                        [-1.4954e-01,  1.7648e-02,  1.0738e-01],
                        [-6.3540e-02,  1.2746e-02,  3.1590e-02]],
              
                       [[-2.2942e-02,  9.1401e-03,  3.2143e-02],
                        [-9.5318e-02, -3.6774e-03,  6.4728e-02],
                        [ 1.7899e-03,  3.2619e-03,  9.8417e-03]]],
              
              
                      [[[ 2.7620e-02,  4.0522e-02,  4.3900e-02],
                        [ 3.0422e-02, -7.2142e-02, -7.3881e-02],
                        [ 3.6847e-02, -1.1538e-01, -8.0408e-02]],
              
                       [[ 4.6463e-02,  5.9469e-02,  7.2776e-02],
                        [ 4.6305e-02

In [4]:
n_2 = 0
for k, v in trained_yolo.items():
    print(k, ' -> ', v.shape)
    n_2 += 1

stage1_conv1.0.weight  ->  torch.Size([32, 3, 3, 3])
stage1_conv1.1.weight  ->  torch.Size([32])
stage1_conv1.1.bias  ->  torch.Size([32])
stage1_conv1.1.running_mean  ->  torch.Size([32])
stage1_conv1.1.running_var  ->  torch.Size([32])
stage1_conv2.0.weight  ->  torch.Size([64, 32, 3, 3])
stage1_conv2.1.weight  ->  torch.Size([64])
stage1_conv2.1.bias  ->  torch.Size([64])
stage1_conv2.1.running_mean  ->  torch.Size([64])
stage1_conv2.1.running_var  ->  torch.Size([64])
stage1_conv3.0.weight  ->  torch.Size([128, 64, 3, 3])
stage1_conv3.1.weight  ->  torch.Size([128])
stage1_conv3.1.bias  ->  torch.Size([128])
stage1_conv3.1.running_mean  ->  torch.Size([128])
stage1_conv3.1.running_var  ->  torch.Size([128])
stage1_conv4.0.weight  ->  torch.Size([64, 128, 1, 1])
stage1_conv4.1.weight  ->  torch.Size([64])
stage1_conv4.1.bias  ->  torch.Size([64])
stage1_conv4.1.running_mean  ->  torch.Size([64])
stage1_conv4.1.running_var  ->  torch.Size([64])
stage1_conv5.0.weight  ->  torch.Size([

In [5]:
yolo = Backbone()

conv_layers = 0
bn_layers = 0
for m in yolo.modules():
    if isinstance(m, torch.nn.Conv2d):
        print(m, ' -> ', m.weight.data.shape)
        conv_layers += 1
    if isinstance(m, torch.nn.BatchNorm2d):
        print(m)
        bn_layers += 1
print(conv_layers, bn_layers)

Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)  ->  torch.Size([32, 3, 3, 3])
BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)  ->  torch.Size([64, 32, 3, 3])
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)  ->  torch.Size([128, 64, 3, 3])
BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)  ->  torch.Size([64, 128, 1, 1])
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)  ->  torch.Size([128, 64, 3, 3])
BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=

In [6]:
l = []
for m in yolo.modules():
    if isinstance(m, torch.nn.Conv2d):
        l.append(m.weight.detach().cpu().numpy())
    if isinstance(m, torch.nn.BatchNorm2d):
        l.append(m.weight.detach().cpu().numpy())
        l.append(m.bias.detach().cpu().numpy())
        l.append(m.running_mean.detach().cpu().numpy())
        l.append(m.running_var.detach().cpu().numpy())
import pickle
pickle.dump(l, open('/home/dk/Desktop/w2.pkl', 'wb'))

In [6]:
trained_keys = list(trained_yolo.keys())
trained_values = list(trained_yolo.values())

idx = 0
for m in yolo.modules():
    if isinstance(m, torch.nn.Conv2d):
        assert m.weight.shape == trained_values[idx].shape
        m.weight.data = trained_values[idx]
        idx += 1
    if isinstance(m, torch.nn.BatchNorm2d):
        assert m.weight.shape == trained_values[idx].shape
        m.weight.data = trained_values[idx]
        idx += 1
        assert m.bias.shape == trained_values[idx].shape
        m.bias.data = trained_values[idx]
        idx += 1
        assert m.running_mean.shape == trained_values[idx].shape
        m.running_mean.data = trained_values[idx]
        idx += 1
        assert m.running_var.shape == trained_values[idx].shape
        m.running_var.data = trained_values[idx]
        idx += 1

In [7]:
trained_yolo_weights_list = []
for k, v in trained_yolo.items():
    trained_yolo_weights_list.append(v.detach().cpu().numpy())

my_yolo_weights_list = []
for m in yolo.modules():
    if isinstance(m, torch.nn.Conv2d):
        my_yolo_weights_list.append(m.weight.detach().cpu().numpy())
    if isinstance(m, torch.nn.BatchNorm2d):
        my_yolo_weights_list.append(m.weight.detach().cpu().numpy())
        my_yolo_weights_list.append(m.bias.detach().cpu().numpy())
        my_yolo_weights_list.append(m.running_mean.detach().cpu().numpy())
        my_yolo_weights_list.append(m.running_var.detach().cpu().numpy())

In [8]:
mse_list = []

for i in range(len(my_yolo_weights_list)):
    mse_list.append(np.mean(np.square(trained_yolo_weights_list[i] - my_yolo_weights_list[i])))

In [9]:
print(mse_list)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [10]:
model_state = yolo.state_dict()

state_dict = {'model': model_state}
torch.save(state_dict, '/home/dk/jyl/Yolo/V2/Notebook/model.pkl')